In [ ]:
import h5py as h5;
import numpy as np;
from scipy import optimize;
import matplotlib.pyplot as plt;

In [ ]:
def SineWave(t, T, Phi=0, A=1, offset=0):
    phi = np.arctan(np.tan(Phi));
    return A*np.sin(2*np.pi*t/T+Phi)+offset;

def CosineWave(t, T, Phi=0, A=1, offset=0):
    phi = np.arctan(np.tan(Phi));
    return A*2*np.pi/T*np.sin(2*np.pi*t/T+Phi);

def GetPhaseDiff( a, b):
    return np.arctan2( np.sin(a-b), np.cos(a-b));

In [ ]:
ch_index = 0
nb_slice = 4000

DY = []
DDY = []

file_prefix = '/sdata/data_challenge1/data_challenge_1a/Data_Challenge_1a_I1_D20200602_T155523_F0'
file_prefix = '/sdata/data_challenge1/data_challenge_1b/Data_Challenge_1b_I1_D20200601_T161750_F00'
#file_prefix = '/sdata/data_challenge1/data_challenge_1b_contrast/Data_Challenge_1b_Finite_I1_D20200607_T163827_F00'

for index in range(1,11):
    
    dump_num = '%02d' % index
    filename = file_prefix + dump_num + '.hdf5'
    file = h5.File( filename )

    #print('Keys:')
    #print( list( file.attrs.keys() ) )
    #print('Keys under adc1:')
    #print( list( file['adc1'].attrs.keys() ) )

    nb_samples = file['adc1'].attrs['nb_samples']
    nb_events = file['adc1'].attrs['nb_events']
    sample_rate = file['adc1'].attrs['sample_rate']

    cal = file['adc1'].attrs['adc_conversion_factor']
    cal_coeff = cal[ch_index][::-1]
    poly = np.poly1d( cal_coeff )


    for evt_index in range(1, nb_events):

        bounds = ([0.1, -np.pi, -2, -0.5], [2, np.pi, 2, 0.5 ]);

        #t1 = 1000*np.linspace( 0, nb_samples/sample_rate, nb_samples)[-nb_slice:]
        y1 = poly(file['adc1']['event_%d' % evt_index][ch_index][-1])
        #par1, opt = optimize.curve_fit(SineWave, t1, y1)

        #t2 = 1000*np.linspace( nb_samples/sample_rate, 2*nb_samples/sample_rate, nb_samples)[:nb_slice]
        y2 = poly(file['adc1']['event_%d' % (evt_index+1)][ch_index][0])
        #par2, opt = optimize.curve_fit(SineWave, t2, y2)

        #t = np.append(t1,t2);

        #dy = (SineWave(t2, *par2)[0]-SineWave(t1, *par1)[-1])/par1[0]
        #ddy = (CosineWave(t2, *par2)[0]-CosineWave(t1, *par1)[-1])
        
        DY = np.append( DY, y2-y1)
        #DDY = np.append( DDY, ddy)
    
    file.close()

In [ ]:
plt.figure( figsize=(12,8) );
plt.hist(DY, bins=np.linspace(-.015, .015, 100));
plt.xlabel("Voltage difference (V)");
plt.ylabel("Count");
plt.title('Data Challenge 1b D20200601 T161750')


In [ ]:
nb_slice = 10

file_prefix = '/sdata/data_challenge1/data_challenge_1b/Data_Challenge_1b_I1_D20200601_T161750_F00'
file_prefix = '/sdata/data_challenge1/data_challenge_1a/Data_Challenge_1a_I1_D20200602_T155523_F0'

index = 2

dump_num = '%03d' % index
filename = file_prefix + dump_num + '.hdf5'
file = h5.File( filename )

nb_samples = file['adc1'].attrs['nb_samples']
nb_events = file['adc1'].attrs['nb_events']
sample_rate = file['adc1'].attrs['sample_rate']

cal = file['adc1'].attrs['adc_conversion_factor']
cal_coeff = cal[ch_index][::-1]
poly = np.poly1d( cal_coeff )


evt_index = 10

t1 = 1000*np.linspace( 0, nb_samples/sample_rate, nb_samples)[-nb_slice:]
y1 = poly(file['adc1']['event_%d' % evt_index][ch_index][-nb_slice:])
        #par1, opt = optimize.curve_fit(SineWave, t1, y1)

t2 = 1000*np.linspace( nb_samples/sample_rate, 2*nb_samples/sample_rate, nb_samples)[:nb_slice]
y2 = poly(file['adc1']['event_%d' % (evt_index+1)][ch_index][:nb_slice])
        #par2, opt = optimize.curve_fit(SineWave, t2, y2)

t = np.append(t1,t2);

plt.plot(t1, y1)
plt.plot(t2, y2)
print(y2[0]-y1[-1:])

In [ ]:
chan_index = 2

event_name = 'event_%d' % evt_index
event = f[event_name][chan_index][0:1000]
plt.plot(event)

In [ ]:
chan_index = 3

event_name = 'event_%d' % evt_index
event = f[event_name][chan_index][0:1000]
plt.plot(event)